In [11]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFileApply
from cognite.client.exceptions import CogniteAPIError, CogniteException

from tests.auth_utils import get_toolkit_client

In [2]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [3]:
SPACE = "sp_upgrade"

In [4]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Existing Cognite TimeSeries with Pending ID

In [5]:
file = FileMetadataWrite(
    external_id="ts_case7_existing_cognite_file",
    name="Case 7 Existing Cognite File with Pending ID",
)

In [6]:
# Ensure the file does not exist before running the test
try:
    client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
    try:
        client.data_modeling.instances.delete(CogniteFileApply(space=SPACE, external_id=file.external_id).as_id())
    except CogniteException as e:
        print(f"\033[91mError\033[0m {e.code} {e.message!s}")
    else:
        print("\033[92mFile node deleted successfully\033[0m")
else:
    print(f"\033[92mFile with external_id {file.external_id} deleted successfully or did not exists\033[0m")

File with external_id ts_case7_existing_cognite_file deleted successfully or did not exists


In [7]:
created, _ = client.files.create(file)
created

,value
external_id,ts_case7_existing_cognite_file
name,Case 7 Existing Cognite File with Pending ID
labels,[]
id,8415190945789795
created_time,2025-07-18 13:05:10.970000
last_updated_time,2025-07-18 13:05:10.970000
uploaded,False


In [8]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name=file.name,
)

In [9]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case7_existing_cognite_file,1,False,2025-07-04 10:12:13.503,2025-07-04 10:12:13.503


In [15]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    if isinstance(e, CogniteAPIError):
        print(f"\033[91mError\033[0m {e.code} {e.message!s}")
    else:
        print(f"\033[91mUnexpected Error\033[0m {e}")

else:
    print("This should not happen, as the file already exists in Cognite.")

Unexpected Error Duplicated: ['sp_upgrade:ts_case7_existing_cognite_file']
The API Failed to process some items.
Successful (2xx): []
Unknown (5xx): []
Failed (4xx): [[{'pendingInstanceId': {'space': 'sp_upgrade', 'externalId': 'ts_case7_existing_cognite_file'}, 'externalId': 'ts_case7_existing_cognite_file'}], ...]
